In [1]:
from os import listdir
from os.path import isfile, join
import numpy as np
import pandas as pd
from IPython.display import display
import datetime
import re
import csv
from geopy.geocoders import Nominatim
from geopy.distance import geodesic # great_circle

# STATIC DATA

comps_data = list()

geolocator = Nominatim(user_agent="hi")

corps_rename_dict = {
    "Vanguard": "Santa Clara Vanguard",
    "Academy": "The Academy",
    "Cavaliers": "The Cavaliers",
    "Bushwackers": "Bushwackers Drum Corps",
    "Connecticut Hurricanes": "Hurricanes",
    "Battalion": "The Battalion",
    "Skyliners": "New York Skyliners"
}

corps_classes = ['DCI World Class', 'DCI Open Class','DCI All Age World Class','DCI All Age Open Class','DCI All Age Class A']

deconstruct = lambda x: " ".join([y for y in x.split(" ") if y != ""])

def export(df: pd.DataFrame, path_no_dot: str, index=True):
    df.to_csv(f'exports/{path_no_dot}.csv', index=index)
    df.to_excel(f'exports/{path_no_dot}.xlsx', index=index)


##################

dict_scores_table = pd.read_excel('new raw data/raw_data.xlsx', sheet_name=None, header=None)

for (k, table) in dict_scores_table.items():

    # general cleaning
    table.loc[1, 0] = "Corps"
    table.loc[1, 1] = "Performance Slot"
    date_and_place_and_comp = table.loc[0, 0]
    table.columns = table.loc[1]
    table.columns = [x.strip() for x in table.columns]
    table.drop([0, 1, 2], inplace=True)

    # date and place
    comp = None
    if len(date_and_place_and_comp.split(sep='--')) > 1:
        (date_and_place, comp) = date_and_place_and_comp.split(sep='--')
    else:
        date_and_place = date_and_place_and_comp
    chunks = date_and_place.split(sep=' ')
    place = " ".join(chunks[1:])

    location = geolocator.geocode(place)
    table["Competition Latitude"] = location.latitude
    table["Competition Longitude"] = location.longitude

    table.insert(0,column='Show Number',value=int(k))
    table.insert(1,column='Date',value=pd.to_datetime(chunks[0]))
    table.insert(2,column='Location',value=place)

    # removing categories
    table.drop([i for (i, row) in table.iterrows() if deconstruct(row['Corps']) in corps_classes], inplace=True)
    
    # type casting
    table['Performance Slot'] = table['Performance Slot'].astype(int)
    table = table.applymap(lambda x: " ".join(x.split()) if isinstance(x, str) else x)

    comps_data.append([place, comp])

scores_table = pd.DataFrame(columns=dict_scores_table['1'].columns)

for (k, v) in dict_scores_table.items():
    scores_table = scores_table.merge(v, how='outer')
    

def whitespace_remover(dataframe):
    for i in dataframe.columns:
        try:
            dataframe[i] = dataframe[i].map(str.strip)
        except:
            pass
# applying whitespace_remover function on dataframe
whitespace_remover(scores_table)


# corps renaming
scores_table['Corps'] = scores_table['Corps'].apply(lambda x: " ".join([y for y in x.split(" ") if y != ""]))
scores_table['Corps'] = scores_table['Corps'].replace(corps_rename_dict)
scores_table['Days from Season Start'] = (scores_table['Date'] - scores_table['Date'].min()).apply(lambda x: x.days)


scores_table = scores_table.applymap(lambda x: re.sub('[^!-~]+',' ',x).strip() if type(x) == str else x)

comps_table = pd.DataFrame(comps_data, columns=['Location', 'DCI Name'])

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(scores_table[scores_table['Show Number'] == 1])
    # display(scores_table[scores_table['Corps'] == "Blue Stars"])


NameError: name 'comps_data' is not defined

In [ ]:
corps_table = pd.read_csv('new raw data/corps_data.csv', index_col='Corps').sort_values(by='Corps')
corps_table.rename(columns=corps_rename_dict, inplace=True)

for corps in corps_table.index:
    scores_table_corps = scores_table.loc[scores_table['Corps'] == corps]
    if scores_table_corps.shape[0] == 0:
        print(corps)
        continue
    scores_table_corps_last = scores_table_corps.iloc[-1,:].to_dict()
    for (k, v) in scores_table_corps_last.items():
        if k != "Corps":
            corps_table.loc[corps,k] = v
    # corps_table.loc[corps,"Average Performace Slot"] = scores_table_corps['Performance Slot'].mean() / comps_table.loc[]

corps_table['Letters in Name'] = [len(x.replace(' ','')) for x in list(corps_table.index)]

# deprecated
corps_table['Latest Score'] = corps_table['Total Score']
corps_table['Last Performance Date'] = corps_table['Date']
corps_table.sort_values(by=['Latest Score','Corps'],inplace=True,ascending=[False, True])

print(corps_table.shape)

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(corps_table.sort_index())

Heat Wave
Les Stentors
(50, 35)


,Home Location,Latitude,Longitude,Conference,Class,Show Number,Date,Location,Performance Slot,General Effect 1,General Effect 2,General Effect Total,Visual Proficiency,Visual Analysis,Visual Color Guard,Visual Total,Music Brass,Music Analysis,Music Percussion,Music Total,Sub Total,Penalty,Total Score,Competition Latitude,Competition Longitude,General Effect 1 1,General Effect 1 2,General Effect 2 1,General Effect 2 2,Music Analysis 1,Music Analysis 2,Days from Season Start,Letters in Name,Latest Score,Last Performance Date
Corps,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
7th Regiment,"New London, CT",41.355619,-72.099780,NaN,DCI Open Class,5.0,2024-06-29,"Shelton, Connecticut",1.0,11.10,11.200,22.300,8.8,12.4,10.9,16.05,8.9,11.100,10.00,15.000,53.350,0.0,53.350,41.320391,-73.083655,NaN,NaN,NaN,NaN,NaN,NaN,3.0,11,53.350,2024-06-29
Atlanta CV,"Atlanta, GA",33.748992,-84.390264,NaN,DCI All Age World Class,23.0,2024-07-09,"Newnan, Georgia",1.0,15.50,15.000,30.500,15.8,15.3,15.2,23.15,16.2,15.400,14.40,23.000,76.650,0.0,76.650,33.380672,-84.799657,NaN,NaN,NaN,NaN,NaN,NaN,13.0,9,76.650,2024-07-09
Blue Devils,"Concord, CA",37.976852,-122.033562,Pacific,DCI World Class,44.0,2024-07-20,"San Antonio, Texas",20.0,18.10,18.525,36.625,18.4,18.0,18.2,27.30,18.7,18.650,18.55,27.950,91.875,0.0,91.875,29.424600,-98.495141,18.2,18.0,18.30,18.75,19.0,18.30,24.0,10,91.875,2024-07-20
Blue Devils B,"Concord, CA",37.976852,-122.033562,NaN,DCI Open Class,35.0,2024-07-14,"Cerritos, California",5.0,14.90,14.700,29.600,15.5,14.7,14.4,22.30,14.5,14.700,14.40,21.800,73.700,0.0,73.700,33.864429,-118.053932,NaN,NaN,NaN,NaN,NaN,NaN,18.0,11,73.700,2024-07-14
Blue Devils C,"Concord, CA",37.976852,-122.033562,NaN,DCI Open Class,18.0,2024-07-07,"Sacramento, California",2.0,11.00,10.400,21.400,9.3,11.3,10.0,15.30,10.2,10.700,10.20,15.550,52.250,0.0,52.250,38.581061,-121.493895,NaN,NaN,NaN,NaN,NaN,NaN,11.0,11,52.250,2024-07-07
Blue Knights,"Denver, CO",39.739236,-104.984862,Southwestern,DCI World Class,44.0,2024-07-20,"San Antonio, Texas",8.0,16.00,15.800,31.800,16.0,15.5,15.0,23.25,15.9,15.900,16.30,24.050,79.100,0.0,79.100,29.424600,-98.495141,16.3,15.7,15.30,16.30,15.8,16.00,24.0,11,79.100,2024-07-20
Blue Stars,"La Crosse, WI",43.812284,-91.251435,Midwestern,DCI World Class,44.0,2024-07-20,"San Antonio, Texas",13.0,16.85,17.225,34.075,16.9,16.7,16.9,25.25,17.4,17.425,16.40,25.613,84.938,0.0,84.938,29.424600,-98.495141,17.0,16.7,17.00,17.45,18.0,16.85,24.0,9,84.938,2024-07-20
Bluecoats,"Canton, OH",40.798546,-81.374951,Eastern,DCI World Class,44.0,2024-07-20,"San Antonio, Texas",21.0,18.45,18.800,37.250,18.6,18.5,18.1,27.60,18.6,18.950,18.80,28.175,93.025,0.0,93.025,29.424600,-98.495141,18.6,18.3,18.70,18.90,19.4,18.50,24.0,9,93.025,2024-07-20
Boston Crusaders,"Boston, MA",42.355433,-71.060511,Eastern,DCI World Class,44.0,2024-07-20,"San Antonio, Texas",19.0,18.10,18.475,36.575,17.9,18.1,18.6,27.30,18.3,18.650,19.00,27.975,91.850,0.0,91.850,29.424600,-98.495141,18.3,17.9,18.50,18.45,19.1,18.20,24.0,15,91.850,2024-07-20


In [ ]:
# determine difference in score by comp

today = datetime.date.today()
day_of_season = (pd.Timestamp(today) - scores_table['Date'].min()).days

growth_fit_xab = lambda x,a,b: (a * x) + b

scores_table_diff = scores_table.copy(deep=True)
scores_table_diff.iloc[:,5:] = 0.0
scores_table_rate = scores_table.copy(deep=True)
scores_table_rate.iloc[:,5:] = 0.0

for corps in set(scores_table['Corps']):
    scores_table_corps = scores_table.loc[scores_table['Corps'] == corps]
    # display(scores_table_corps)
    scores_table_diff_corps = scores_table_diff.loc[scores_table_diff['Corps'] == corps]
    scores_table_rate_corps = scores_table_rate.loc[scores_table_rate['Corps'] == corps]
    
    for c in scores_table_corps.columns:
        if c not in ['Corps','Location']:
            scores_table_diff_corps.loc[:,c] = scores_table_corps.loc[:,c].diff()
    
    for c in scores_table_corps.columns:
        if c not in ['Corps','Location']:
            scores_table_rate_corps.loc[:,c] = scores_table_diff_corps.loc[:,c] / scores_table_diff_corps.loc[:,"Date"].apply(lambda x: x.days)

    for (i, r) in scores_table_diff_corps.iterrows():
        scores_table_diff.iloc[i,:] = r
    
    for (i, r) in scores_table_rate_corps.iterrows():
        scores_table_rate.iloc[i,:] = r
    
    # WIP CODE RIGHT HERE
    # display(scores_table_corps)
    scores_table_corps_shifted = scores_table_corps.shift(1)
    # display(scores_table_corps_shifted)

    for (i, r) in scores_table_corps_shifted.iterrows():
        if scores_table_corps_shifted.index[0] != i:
            geodesic_distance = geodesic((scores_table_corps.loc[i, "Competition Latitude"], scores_table_corps.loc[i, "Competition Longitude"]), (r["Competition Latitude"], r["Competition Longitude"])).mi
            scores_table_diff.loc[i, "Geodesic Distance"] = geodesic_distance
    
    corps_table.loc[corps, "Average Geodesic Distance"] = scores_table_diff[scores_table_diff['Corps'] == corps]["Geodesic Distance"].mean()
    corps_table.loc[corps, "Total Geodesic Distance"] = scores_table_diff[scores_table_diff['Corps'] == corps]["Geodesic Distance"].sum()
    corps_table.loc[corps, "Geodesic Distance Per Day"] = corps_table.loc[corps, "Total Geodesic Distance"] / scores_table_corps.shape[0]

    # display(scores_table_diff_corps)

    if int(scores_table_corps.shape[0]) > 2: # must have more than 2 data points lowkey
        ((a, b), cov) = np.polyfit(scores_table_corps['Days from Season Start'].astype(int), scores_table_corps['Total Score'].astype(float), 1, cov=True)
        (sa, sb) = np.sqrt(np.diag(cov))
        corps_table.loc[corps, "RMSE"] = (scores_table_corps['Total Score'] - scores_table_corps['Days from Season Start'].apply(lambda x: growth_fit_xab(x, a, b))).std()

        for i in range(1,7+1):
            est = growth_fit_xab(day_of_season + i, a + (sa * 0), b + (sb * 0))
            upper = growth_fit_xab(day_of_season + i, a + (sa * i), b + (sb * i))
            lower = growth_fit_xab(day_of_season + i, a + (sa * -i), b + (sb * -i))
            est = 100.0 if est > 100.0 else est
            upper = 100.0 if upper > 100.0 else upper
            lower = 100.0 if lower > 100.0 else lower
            corps_table.loc[corps, f"D+{i}"] = 0.0 if est < 0.0 else est
            corps_table.loc[corps, f"D+{i} U"] = 0.0 if upper < 0.0 else upper
            corps_table.loc[corps, f"D+{i} L"] = 0.0 if lower < 0.0 else lower
    
    corps_table.loc[corps,'Number of Shows'] = int(scores_table_corps.shape[0])
    corps_table.loc[corps, 'Average Rest'] = scores_table_diff_corps.loc[:,'Date'].apply(lambda x: x.days).mean()
    corps_table.loc[corps, 'Average Score Improvement'] = scores_table_rate_corps.loc[:,'Total Score'].mean()
    
    # normal ranges from 0 to 1
    corps_table.loc[corps,'Average of Last 3 Scores'] = scores_table_corps['Total Score'].iloc[-3:].mean()
    corps_table.loc[corps,'Average Normal General Effect'] = scores_table_corps['General Effect Total'].iloc[-3:].mean() / 40
    corps_table.loc[corps,'Average Normal Visual'] = scores_table_corps['Visual Total'].iloc[-3:].mean() / 30
    corps_table.loc[corps,'Average Normal Music'] = scores_table_corps['Music Total'].iloc[-3:].mean() / 30
    corps_table.loc[corps,'Average Normal Brass'] = scores_table_corps['Music Brass'].iloc[-3:].mean() / 20
    corps_table.loc[corps,'Average Normal Percussion'] = scores_table_corps['Music Percussion'].iloc[-3:].mean() / 20

    if len(set(corps_table.loc[corps, ['Average Normal General Effect', 'Average Normal Visual', 'Average Normal Music']].values)) < len(list(corps_table.loc[corps, ['Average Normal General Effect', 'Average Normal Visual', 'Average Normal Music']].values)):
        corps_table.loc[corps, 'Best Caption'] = 'Multiple'
    else:
        max_column_index = np.argmax(corps_table.loc[corps, ['Average Normal General Effect', 'Average Normal Visual', 'Average Normal Music']].values)
        corps_table.loc[corps, 'Best Caption'] = ['General Effect', 'Visual', 'Music'][max_column_index]


    corps_table.loc[corps, 'Best Music'] = ['Brass', 'Percussion'][int(corps_table.loc[corps, 'Average Normal Brass'] < corps_table.loc[corps, 'Average Normal Percussion'])] if corps_table.loc[corps, 'Average Normal Brass'] != corps_table.loc[corps, 'Average Normal Percussion'] else 'Neither'

    # with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    #     if corps == "Phantom Regiment":
    #         display(scores_table_corps)
    #         display(scores_table_diff_corps)
    #         display(scores_table_rate_corps)

corps_table['Rank of Last 3 Scores'] = corps_table['Average of Last 3 Scores'].rank(ascending=False, method='max')
corps_table['Rank of Last Score'] = corps_table['Latest Score'].rank(ascending=False, method='max')
corps_table['Rank of Last 3 Scores No All-Age'] = corps_table.loc[['All Age' not in str(x) for x in corps_table['Class']], 'Average of Last 3 Scores'].rank(ascending=False, method='max')
corps_table['Rank of Last Score No All-Age'] = corps_table.loc[['All Age' not in str(x) for x in corps_table['Class']], 'Latest Score'].rank(ascending=False, method='max')

def ordinal(n):
    if str(n) == 'nan':
        return None
    else:
        return "%d%s" % (int(n),"tsnrhtdd"[(int(n)//10%10!=1)*(int(n)%10<4)*int(n)%10::4])
    
corps_table['Ordinal Rank of Last 3 Scores'] = corps_table['Rank of Last 3 Scores'].apply(ordinal)
corps_table['Ordinal Rank of Last Score'] = corps_table['Rank of Last Score'].apply(ordinal)
corps_table['Ordinal Rank of Last 3 Scores No All-Age'] = corps_table['Rank of Last 3 Scores No All-Age'].apply(ordinal)
corps_table['Ordinal Rank of Last Score No All-Age'] = corps_table['Rank of Last Score No All-Age'].apply(ordinal)


championship_rounds_table = pd.DataFrame([None,'Prelims','Semis','Finals'],columns=['Championship Rounds'])

def calc_championship_round(rank: float):
    if rank <= 12:
        return championship_rounds_table['Championship Rounds'][3]
    elif rank <= 25:
        return championship_rounds_table['Championship Rounds'][2]
    elif rank <= 40:
        return championship_rounds_table['Championship Rounds'][1]
    else:
        return championship_rounds_table['Championship Rounds'][0]

corps_table['Projected Championship Round'] = corps_table['Rank of Last 3 Scores No All-Age'].apply(calc_championship_round)

captions_table = pd.DataFrame(['General Effect','Visual','Music','Multiple','Brass','Percussion','Neither'],columns=['Caption Title'])

# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    # display(corps_table)
    # display(scores_table_diff)

In [ ]:
# calculate placements per comp per group

# scores_table = scores_table.copy(deep=True)
set_show_numbers = set(scores_table['Show Number'])
dict_corps_table_class = corps_table['Class'].to_dict()

for show_number in set_show_numbers:
    scores_table_show = scores_table[scores_table['Show Number'] == show_number].copy(deep=True)
    scores_table_show.loc[:,'Class'] = scores_table_show['Corps'].map(dict_corps_table_class)

    if show_number == 45:
        display(scores_table_show)

    for performance_class in set(scores_table_show['Class']):
        scores_table_show_class = scores_table_show[scores_table_show['Class'] == performance_class].copy(deep=True)
        scores_table_show_class.sort_values('Total Score', ascending=False, inplace=True)
        
        scores_table_show_class['Show and Class Placement'] = scores_table_show_class['Total Score'].rank(ascending=False, method='min')
        set_placement = scores_table_show_class['Show and Class Placement'].astype(int).to_dict()

        scores_table_show_class['Show and Class Points'] = scores_table_show_class['Total Score'].rank(ascending=True, method='max')
        set_points = scores_table_show_class['Show and Class Points'].astype(int).to_dict()

        for (k, v) in set_placement.items():
            scores_table.loc[k,'Show and Class Placement'] = v

        for (k, v) in set_points.items():
            scores_table.loc[k,'Show and Class Points'] = v


with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(scores_table)

scores_table['Show and Class Placement'] = scores_table['Show and Class Placement'].astype(int)
scores_table['Show and Class Points'] = scores_table['Show and Class Points'].astype(int)

max_placement = scores_table['Show and Class Placement'].max()

corps_table['Participate Place'] = 0
for (corps, row) in corps_table.iterrows():
    for i in range(1, max_placement + 1):
        corps_table.loc[corps, f'{i} Place'] = scores_table['Show and Class Placement'][scores_table['Show and Class Placement'] == i][scores_table['Corps'] == corps].count()
        
    corps_table.loc[corps, 'Participate Place'] = corps_table.loc[corps, [f'{i} Place' for i in range(3 + 1, max_placement + 1)]].sum()

    corps_table.loc[corps, 'Total Points'] = scores_table['Show and Class Points'][scores_table['Corps'] == corps].sum()

corps_table['Total Points'] = corps_table['Total Points'].astype(int)

for i in range(1, max_placement + 1):
    corps_table[f'{i} Place'] = corps_table[f'{i} Place'].astype(int)

corps_table_sorted_placement = corps_table.sort_values(by=[f'{i} Place' for i in range(1, max_placement + 1)], ascending=[False for _ in range(1, max_placement + 1)]).reset_index().copy(deep=True)
corps_table_sorted_placement['Placement Rank'] = corps_table_sorted_placement.index + 1

dict_placement = corps_table_sorted_placement.set_index("Corps")['Placement Rank'].to_dict()

for (k, v) in dict_placement.items():
    corps_table.loc[k, 'Placement Rank'] = v
display(scores_table[scores_table['Corps'] == 'Phantom Regiment'])
display(corps_table)

,Show Number,Date,Location,Corps,Performance Slot,General Effect 1,General Effect 2,General Effect Total,Visual Proficiency,Visual Analysis,...,General Effect 1 1,General Effect 1 2,General Effect 2 1,General Effect 2 2,Music Analysis 1,Music Analysis 2,Days from Season Start,Show and Class Placement,Show and Class Points,Class
298,45,2024-07-20,"Landisville, Pennsylvania",Raiders,1,12.4,12.7,25.1,12.8,12.7,...,NaN,NaN,NaN,NaN,NaN,NaN,24,NaN,NaN,DCI Open Class
299,45,2024-07-20,"Landisville, Pennsylvania",Reading Buccaneers,3,17.8,17.7,35.5,17.2,17.8,...,NaN,NaN,NaN,NaN,NaN,NaN,24,NaN,NaN,DCI All Age World Class
300,45,2024-07-20,"Landisville, Pennsylvania",Bushwackers Drum Corps,2,17.3,17.1,34.4,16.7,17.3,...,NaN,NaN,NaN,NaN,NaN,NaN,24,NaN,NaN,DCI All Age World Class
301,45,2024-07-20,"Landisville, Pennsylvania",Hawthorne Caballeros,1,17.1,16.8,33.9,16.9,17.0,...,NaN,NaN,NaN,NaN,NaN,NaN,24,NaN,NaN,DCI All Age World Class
302,45,2024-07-20,"Landisville, Pennsylvania",White Sabers,1,15.4,15.6,31.0,15.8,15.2,...,NaN,NaN,NaN,NaN,NaN,NaN,24,NaN,NaN,DCI All Age Open Class
303,45,2024-07-20,"Landisville, Pennsylvania",Fusion Core,2,15.1,15.2,30.3,15.5,14.9,...,NaN,NaN,NaN,NaN,NaN,NaN,24,NaN,NaN,DCI All Age Open Class
304,45,2024-07-20,"Landisville, Pennsylvania",New York Skyliners,1,13.6,13.2,26.8,13.5,13.7,...,NaN,NaN,NaN,NaN,NaN,NaN,24,NaN,NaN,DCI All Age Class A


,Show Number,Date,Location,Corps,Performance Slot,General Effect 1,General Effect 2,General Effect Total,Visual Proficiency,Visual Analysis,Visual Color Guard,Visual Total,Music Brass,Music Analysis,Music Percussion,Music Total,Sub Total,Penalty,Total Score,Competition Latitude,Competition Longitude,General Effect 1 1,General Effect 1 2,General Effect 2 1,General Effect 2 2,Music Analysis 1,Music Analysis 2,Days from Season Start,Show and Class Placement,Show and Class Points
0,1,2024-06-26,"Rockford, Michigan",Boston Crusaders,5,15.40,14.400,29.800,14.5,14.3,14.5,21.65,14.3,14.700,14.10,21.550,73.000,0,73.000,43.119721,-85.559604,NaN,NaN,NaN,NaN,NaN,NaN,0,1.0,5.0
1,1,2024-06-26,"Rockford, Michigan",Phantom Regiment,2,14.80,14.200,29.000,14.0,13.6,13.6,20.60,13.8,14.200,13.10,20.550,70.150,0,70.150,43.119721,-85.559604,NaN,NaN,NaN,NaN,NaN,NaN,0,2.0,4.0
2,1,2024-06-26,"Rockford, Michigan",The Cavaliers,4,13.70,13.900,27.600,13.8,13.3,12.9,20.00,13.4,14.100,13.80,20.650,68.250,0,68.250,43.119721,-85.559604,NaN,NaN,NaN,NaN,NaN,NaN,0,3.0,3.0
3,1,2024-06-26,"Rockford, Michigan",Blue Stars,1,14.10,13.300,27.400,13.3,12.8,13.4,19.75,12.9,13.600,12.50,19.500,66.650,0,66.650,43.119721,-85.559604,NaN,NaN,NaN,NaN,NaN,NaN,0,4.0,2.0
4,1,2024-06-26,"Rockford, Michigan",Colts,3,13.80,13.000,26.800,13.4,12.6,12.8,19.40,12.7,13.300,12.40,19.200,65.400,0,65.400,43.119721,-85.559604,NaN,NaN,NaN,NaN,NaN,NaN,0,5.0,1.0
5,2,2024-06-28,"Muncie, Indiana",Carolina Crown,6,14.90,15.400,30.300,15.2,15.0,14.6,22.40,15.1,15.100,14.20,22.200,74.900,0,74.900,40.193689,-85.386527,NaN,NaN,NaN,NaN,NaN,NaN,2,1.0,6.0
6,2,2024-06-28,"Muncie, Indiana",Boston Crusaders,5,15.00,15.000,30.000,14.7,15.2,14.9,22.40,14.8,14.800,15.00,22.300,74.700,0,74.700,40.193689,-85.386527,NaN,NaN,NaN,NaN,NaN,NaN,2,2.0,5.0
7,2,2024-06-28,"Muncie, Indiana",Phantom Regiment,3,14.50,14.700,29.200,14.5,14.2,13.8,21.25,14.4,14.700,14.30,21.700,72.150,0,72.150,40.193689,-85.386527,NaN,NaN,NaN,NaN,NaN,NaN,2,3.0,4.0
8,2,2024-06-28,"Muncie, Indiana",The Cavaliers,2,13.70,14.100,27.800,14.0,13.4,13.1,20.25,13.7,14.300,14.10,21.050,69.100,0,69.100,40.193689,-85.386527,NaN,NaN,NaN,NaN,NaN,NaN,2,4.0,3.0
9,2,2024-06-28,"Muncie, Indiana",Blue Stars,1,13.40,13.800,27.200,13.2,13.3,13.5,20.00,13.3,13.800,12.90,20.000,67.200,0,67.200,40.193689,-85.386527,NaN,NaN,NaN,NaN,NaN,NaN,2,5.0,2.0


C:\Users\justr\AppData\Local\Temp\ipykernel_44352\3134831042.py:59: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`



,Show Number,Date,Location,Corps,Performance Slot,General Effect 1,General Effect 2,General Effect Total,Visual Proficiency,Visual Analysis,...,Competition Longitude,General Effect 1 1,General Effect 1 2,General Effect 2 1,General Effect 2 2,Music Analysis 1,Music Analysis 2,Days from Season Start,Show and Class Placement,Show and Class Points
1,1,2024-06-26,"Rockford, Michigan",Phantom Regiment,2,14.80,14.20,29.0,14.0,13.6,...,-85.559604,NaN,NaN,NaN,NaN,NaN,NaN,0,2,4
7,2,2024-06-28,"Muncie, Indiana",Phantom Regiment,3,14.50,14.70,29.2,14.5,14.2,...,-85.386527,NaN,NaN,NaN,NaN,NaN,NaN,2,3,4
44,7,2024-06-30,"Lisle, Illinois",Phantom Regiment,4,15.00,14.70,29.7,15.0,14.5,...,-88.074769,NaN,NaN,NaN,NaN,NaN,NaN,4,3,5
62,9,2024-07-02,"Mason, OH",Phantom Regiment,1,15.60,15.00,30.6,15.2,14.7,...,-84.309939,NaN,NaN,NaN,NaN,NaN,NaN,6,4,1
77,13,2024-07-05,"Rockford, Illinois",Phantom Regiment,4,15.70,15.30,31.0,15.6,15.6,...,-89.093966,NaN,NaN,NaN,NaN,NaN,NaN,9,1,4
99,16,2024-07-06,"Whitewater, Wisconsin",Phantom Regiment,2,15.90,15.80,31.7,15.8,15.9,...,-88.729268,NaN,NaN,NaN,NaN,NaN,NaN,10,2,4
106,17,2024-07-07,"LaCrosse, Wisconsin",Phantom Regiment,2,15.80,16.90,32.7,16.0,16.1,...,-91.104106,NaN,NaN,NaN,NaN,NaN,NaN,11,2,4
134,22,2024-07-09,"Mankato, Minnesota",Phantom Regiment,3,16.30,16.40,32.7,16.1,16.6,...,-93.999351,NaN,NaN,NaN,NaN,NaN,NaN,13,1,3
155,25,2024-07-11,"Dubuque, Iowa",Phantom Regiment,3,16.40,16.60,33.0,16.5,16.7,...,-90.664799,NaN,NaN,NaN,NaN,NaN,NaN,15,1,4
195,31,2024-07-13,"DeKalb, Illinois",Phantom Regiment,6,16.80,17.00,33.8,16.9,16.9,...,-88.771395,NaN,NaN,NaN,NaN,NaN,NaN,17,2,5


,Home Location,Latitude,Longitude,Conference,Class,Show Number,Date,Location,Performance Slot,General Effect 1,...,14 Place,15 Place,16 Place,17 Place,18 Place,19 Place,20 Place,21 Place,Total Points,Placement Rank
Corps,,,,,,,,,,,,,,,,,,,,,
Bluecoats,"Canton, OH",40.798546,-81.374951,Eastern,DCI World Class,44.0,2024-07-20,"San Antonio, Texas",21.0,18.45,...,0,0,0,0,0,0,0,0,66,1.0
Blue Devils,"Concord, CA",37.976852,-122.033562,Pacific,DCI World Class,44.0,2024-07-20,"San Antonio, Texas",20.0,18.10,...,0,0,0,0,0,0,0,0,76,2.0
Boston Crusaders,"Boston, MA",42.355433,-71.060511,Eastern,DCI World Class,44.0,2024-07-20,"San Antonio, Texas",19.0,18.10,...,0,0,0,0,0,0,0,0,64,7.0
Carolina Crown,"Fort Mill, SC",35.007370,-80.945076,Eastern,DCI World Class,44.0,2024-07-20,"San Antonio, Texas",17.0,17.65,...,0,0,0,0,0,0,0,0,56,11.0
Phantom Regiment,"Rockford, IL",42.271394,-89.093966,Midwestern,DCI World Class,44.0,2024-07-20,"San Antonio, Texas",18.0,17.65,...,0,0,0,0,0,0,0,0,67,8.0
Reading Buccaneers,"Reading, PA",40.335345,-75.927949,NaN,DCI All Age World Class,45.0,2024-07-20,"Landisville, Pennsylvania",3.0,17.80,...,0,0,0,0,0,0,0,0,11,12.0
Santa Clara Vanguard,"Santa Clara, CA",37.233325,-121.684635,Pacific,DCI World Class,44.0,2024-07-20,"San Antonio, Texas",16.0,17.35,...,0,0,0,0,0,0,0,0,58,21.0
Mandarins,"Sacramento, CA",38.581061,-121.493895,Pacific,DCI World Class,44.0,2024-07-20,"San Antonio, Texas",15.0,17.30,...,0,0,0,0,0,0,0,0,57,20.0
Bushwackers Drum Corps,"Princeton, NJ",40.349695,-74.659738,NaN,DCI All Age World Class,45.0,2024-07-20,"Landisville, Pennsylvania",2.0,17.30,...,0,0,0,0,0,0,0,0,8,34.0


In [ ]:
from pybaseball import standings

# mlb_table = pd.read_csv('mlb/mlb.csv')
# display(mlb_table)

data = standings()

mlb_table = pd.concat(data, axis=0).sort_values('W-L%',ascending=False).reset_index(drop=True)
mlb_table.index = mlb_table.index + 1

display(mlb_table)

,Tm,W,L,W-L%,GB
1,Philadelphia Phillies,63,37,.630,--
2,Baltimore Orioles,60,39,.606,--
3,Cleveland Guardians,59,40,.596,--
4,Los Angeles Dodgers,60,41,.594,--
5,New York Yankees,60,42,.588,1.5
6,Milwaukee Brewers,57,43,.570,--
7,Minnesota Twins,55,44,.556,4.0
8,Kansas City Royals,56,45,.554,4.0
9,Atlanta Braves,54,45,.545,8.5
10,Boston Red Sox,53,46,.535,7.0


In [ ]:
import plotly.graph_objects as go
import plotly.colors as pc

(a, b) = np.polyfit(scores_table['Days from Season Start'].astype(int), scores_table['Total Score'].astype(float), 1)
growth_fit = lambda x: (a * x) + b

scores_table['Total Score Residual'] = scores_table['Total Score'] - scores_table['Days from Season Start'].apply(growth_fit)

fig = go.Figure()

# adding player
ranking = 1
trace_colors = pc.qualitative.Bold
for (i, corps) in enumerate(corps_table.index):
    scores_table_corps = scores_table.loc[scores_table['Corps'] == corps]
    scores_table_diff_corps = scores_table_diff.loc[scores_table['Corps'] == corps]
    scores_table_rate_corps = scores_table_rate.loc[scores_table['Corps'] == corps]
    fig.add_trace(go.Scatter(
        x=scores_table_corps['Date'],
        y=scores_table_corps['Total Score'],
        name=f'#{ranking} ({corps_table.loc[corps,"Latest Score"]:.3f}) {corps}',
        mode='lines+markers',
        connectgaps=True,
        line=dict(
            #shape='hv',
            color=trace_colors[i % len(trace_colors)]
        ),
        text=[
            f'<b>{x[0]}</b><br><br>' + 
            f"{x[1].strftime('%A, %d %B %Y')}<br>"
            f"{x[2]}<br><br>" +
            f"Competition Score: {x[3]:.3f}<br>"
            f"Difference: {x[4]:+.3f}<br>"
            f"Score Growth: {x[5]:+.3f} per day"
            for x in zip(
                scores_table_corps['Corps'],
                scores_table_corps['Date'],
                scores_table_corps['Location'],
                scores_table_corps['Total Score'],
                scores_table_diff_corps['Total Score'],
                scores_table_rate_corps['Total Score']
            )
        ]
    ))
    ranking += 1

updated_time = f'<i>Updated {str(datetime.datetime.now().strftime("%A, %b %d, %Y %H:%M:%S"))} CT</i>'

def active_ranking(cat):
    i = 0
    t = 0
    retList = list()
    for c in cat:
        retList.append(f"#{i+1} ({corps_table['Latest Score'][t]:.3f}) {corps_table.index[t]}")
        if c is True:
            i += 1
        t += 1
    return retList 

dropdown_labels = ["All Corps"] + corps_classes

dropdown_categories = [[True for _ in corps_table['Class']]] + [
    [x == corps_classes[i] for x in corps_table['Class']] for i in range(len(corps_classes))
]

dropdown_dicts = [
    dict(
    label=label,
    method='restyle',
    args=[{"visible": category, "name": active_ranking(category)}]
    ) for (label, category) in zip(dropdown_labels, dropdown_categories)
]


fig.update_layout(
    title=f'<b>DCI 2024 Scores by Roman Ramirez</b><br>{updated_time}<br>',
    xaxis_title='<b>Date</b>',
    yaxis_title='<b>Total Score</b>',
    updatemenus=[
        dict(
            active=0,
            buttons=dropdown_dicts
            ,
        )       
    ]
)

customdata = np.stack((
    list(scores_table['Location']),
    list(scores_table_diff['Total Score']),
    list(scores_table['Corps'])
    ), axis=0)
hovertemplate = (
    '%{text}<br>' +
    '<extra></extra>'
)

fig.update_traces(
    customdata=customdata,
    hovertemplate=hovertemplate,
    opacity=0.8,
    legendgrouptitle_text='<b>#<i>Rank</i> (<i>Latest Score</i>) <i>Corps</i></b>'
)

fig.show()
fig.write_html("index.html")

export(scores_table, 'scores', True)
export(scores_table_diff, 'scores_diff', True)
export(scores_table_rate, 'scores_rate', True)
export(corps_table, 'corps', True)
export(comps_table, 'comps', True)
export(captions_table, 'captions', True)
export(championship_rounds_table, 'championship_rounds', True)
export(mlb_table, 'mlb', True)

#initialze the excel writer
writer = pd.ExcelWriter('exports/dci_data.xlsx', engine='xlsxwriter')

#store your dataframes in a  dict, where the key is the sheet name you want
frames = {
    'scores': scores_table, 
    'scores_diff': scores_table_diff,
    'scores_rate': scores_table_rate,
    'corps': corps_table, 
    'comps': comps_table, 
    'captions': captions_table, 
    'championship rounds': championship_rounds_table, 
    'mlb': mlb_table
    }

#now loop thru and put each on a specific sheet
for sheet, frame in  frames.items(): # .use .items for python 3.X
    frame.to_excel(writer, sheet_name = sheet)

#critical last step
writer.close()

NameError: name 'comps_table' is not defined

In [ ]:
#to dos

# in import, create "static" corps master file
# # contains class, and other corps-related info

# SQL join this on scores table
# add buttons to filter by class